# Проект: Класифікація зображень за допомогою CNN

Система автоматизованої обробки та класифікації зображень


## Блок 1: Імпорт необхідних бібліотек

**Що це:** Підключення всіх потрібних інструментів для роботи  
**Навіщо:** Без цих бібліотек не можна працювати з даними, моделями та зображеннями


In [ ]:
# Основні бібліотеки для роботи з даними
import numpy as np
import matplotlib.pyplot as plt

# Бібліотеки для роботи з нейронними мережами
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

# Бібліотеки для обробки зображень
from PIL import Image
import cv2

# Бібліотеки для оцінки моделі
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print("TensorFlow version:", tf.__version__)
print("Всі бібліотеки успішно завантажено!")


## Блок 2: Завантаження та огляд датасету

**Що це:** Завантаження готового датасету CIFAR-10 (10 класів зображень)  
**Навіщо:** Нам потрібні зображення для навчання моделі (мінімум 100 прикладів)


In [ ]:
# Завантаження датасету CIFAR-10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Назви класів
class_names = ['Літак', 'Автомобіль', 'Птах', 'Кіт', 'Олень', 
               'Собака', 'Жаба', 'Кінь', 'Корабель', 'Вантажівка']

# Виведення інформації про дані
print("Форма навчальних даних:", x_train.shape)
print("Форма тестових даних:", x_test.shape)
print("Кількість класів:", len(class_names))
print("\nПерші кілька зображень:")

# Візуалізація перших 10 зображень
plt.figure(figsize=(15, 3))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(x_train[i])
    plt.title(f"Клас: {class_names[y_train[i][0]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()


## Блок 3: Попередня обробка даних

**Що це:** Нормалізація та форматування даних для навчання  
**Навіщо:** Модель працює краще, коли дані в діапазоні 0-1 та правильно форматувані


In [ ]:
# Нормалізація пікселів (перетворення з 0-255 в 0-1)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Перетворення міток у категорійний формат (one-hot encoding)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Створення валідаційної вибірки (10% від навчальної)
x_val = x_train[45000:]
y_val = y_train[45000:]
x_train = x_train[:45000]
y_train = y_train[:45000]

print("Розмір навчальної вибірки:", x_train.shape[0])
print("Розмір валідаційної вибірки:", x_val.shape[0])
print("Розмір тестової вибірки:", x_test.shape[0])
print("\nДані готові для навчання!")


## Блок 4: Створення архітектури CNN моделі

**Що це:** Проектування структури згорткової нейронної мережі  
**Навіщо:** CNN добре розпізнає об'єкти на зображеннях, Batch Normalization запобігає перенавчанню


In [ ]:
# Створення моделі CNN
model = Sequential([
    # Перший згортковий шар
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'),
    BatchNormalization(),  # Регуляризація
    MaxPooling2D((2, 2)),
    
    # Другий згортковий шар
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),  # Регуляризація
    MaxPooling2D((2, 2)),
    
    # Третій згортковий шар
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),  # Регуляризація
    MaxPooling2D((2, 2)),
    
    # Перетворення 2D в 1D
    Flatten(),
    
    # Повнозв'язний шар
    Dense(256, activation='relu'),
    Dropout(0.5),  # Запобігання перенавчанню
    
    # Вихідний шар (10 класів)
    Dense(10, activation='softmax')
])

# Компіляція моделі
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),  # Оптимізатор
    loss='categorical_crossentropy',  # Функція втрат
    metrics=['accuracy']  # Метрика для оцінки
)

# Виведення структури моделі
model.summary()


## Блок 5: Навчання моделі

**Що це:** Процес навчання моделі на навчальних даних  
**Навіщо:** Модель вчиться розпізнавати об'єкти через багато епох (проходів по даних)


In [ ]:
# Налаштування параметрів навчання
epochs = 20  # Кількість епох
batch_size = 32  # Розмір пакету

# Запуск навчання
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_val, y_val),
    verbose=1
)

print("\nНавчання завершено!")


## Блок 6: Візуалізація процесу навчання

**Що це:** Графіки показують, як покращується модель  
**Навіщо:** Допомагає зрозуміти, чи добре навчається модель та чи немає перенавчання


In [ ]:
# Графік точності
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Точність на навчанні')
plt.plot(history.history['val_accuracy'], label='Точність на валідації')
plt.xlabel('Епоха')
plt.ylabel('Точність')
plt.legend()
plt.title('Точність моделі')

# Графік втрат
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Втрати на навчанні')
plt.plot(history.history['val_loss'], label='Втрати на валідації')
plt.xlabel('Епоха')
plt.ylabel('Втрати')
plt.legend()
plt.title('Втрати моделі')

plt.tight_layout()
plt.show()


## Блок 7: Оцінка моделі на тестових даних

**Що це:** Перевірка якості моделі на нових даних  
**Навіщо:** Оцінюємо, чи модель добре працює на зображеннях, які вона не бачила під час навчання


In [ ]:
# Оцінка моделі на тестових даних
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)

print(f"Точність на тестових даних: {test_accuracy*100:.2f}%")
print(f"Втрати на тестових даних: {test_loss:.4f}")

# Передбачення на тестових даних
y_pred = model.predict(x_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Детальний звіт про метрики
print("\n" + "="*50)
print("ДЕТАЛЬНИЙ ЗВІТ ПРО ЯКІСТЬ МОДЕЛІ")
print("="*50)
print(classification_report(y_test_classes, y_pred_classes, 
                          target_names=class_names))


## Блок 8: Матриця плутанини (Confusion Matrix)

**Що це:** Візуалізація помилок класифікації  
**Навіщо:** Показує, які класи плутає модель та наскільки точно вона працює


In [ ]:
# Створення матриці плутанини
cm = confusion_matrix(y_test_classes, y_pred_classes)

# Візуалізація
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Передбачений клас')
plt.ylabel('Реальний клас')
plt.title('Матриця плутанини')
plt.tight_layout()
plt.show()


## Блок 9: Тестування на окремих зображеннях

**Що це:** Перевірка роботи моделі на конкретних прикладах  
**Навіщо:** Демонструє, як модель розпізнає об'єкти на реальних зображеннях


In [ ]:
# Вибір 16 випадкових зображень з тестового набору
num_images = 16
indices = np.random.choice(len(x_test), num_images, replace=False)

plt.figure(figsize=(15, 10))
for i, idx in enumerate(indices):
    plt.subplot(4, 4, i+1)
    
    # Зображення
    image = x_test[idx]
    plt.imshow(image)
    
    # Передбачення
    prediction = model.predict(np.expand_dims(image, axis=0), verbose=0)
    predicted_class = np.argmax(prediction[0])
    confidence = prediction[0][predicted_class] * 100
    
    # Реальний клас
    true_class = np.argmax(y_test[idx])
    
    # Колір тексту (зелений якщо правильно, червоний якщо ні)
    color = 'green' if predicted_class == true_class else 'red'
    
    plt.title(f"Реальний: {class_names[true_class]}\n"
              f"Передбачено: {class_names[predicted_class]} ({confidence:.1f}%)",
              color=color, fontsize=8)
    plt.axis('off')

plt.tight_layout()
plt.show()


## Блок 10: Вимірювання швидкості обробки

**Що це:** Перевірка часу обробки одного зображення  
**Навіщо:** Вимога: час обробки не більше 0.5 секунди на зображення


In [ ]:
import time

# Вимірювання часу обробки 100 зображень
num_samples = 100
test_images = x_test[:num_samples]

start_time = time.time()
predictions = model.predict(test_images, verbose=0)
end_time = time.time()

total_time = end_time - start_time
time_per_image = total_time / num_samples

print(f"Час обробки {num_samples} зображень: {total_time:.4f} секунд")
print(f"Середній час обробки одного зображення: {time_per_image:.4f} секунд")

if time_per_image <= 0.5:
    print(f"\n✓ Вимога виконана! (≤ 0.5 секунди)")
else:
    print(f"\n✗ Вимога не виконана (потрібно ≤ 0.5 секунди)")


## Блок 11: Покращення якості зображень - Підвищення яскравості

**Що це:** Обробка зображення для покращення його вигляду  
**Навіщо:** Демонстрація обробки зображень (стилізація, колоризація, реставрація)


In [ ]:
# Вибір зображення для обробки
sample_image = x_test[0] * 255  # Повертаємо до формату 0-255
sample_image = sample_image.astype(np.uint8)

# Покращення яскравості та контрасту
enhanced_image = cv2.convertScaleAbs(sample_image, alpha=1.5, beta=30)

# Застосування фільтра для зменшення шуму
denoised_image = cv2.fastNlMeansDenoisingColored(sample_image, None, 10, 10, 7, 21)

# Візуалізація результатів
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(sample_image)
plt.title('Оригінальне зображення')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(enhanced_image)
plt.title('Підвищена яскравість та контраст')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(denoised_image)
plt.title('Зменшення шуму')
plt.axis('off')

plt.tight_layout()
plt.show()

print("Приклади обробки зображень завершено!")


## Блок 12: Збереження моделі

**Що це:** Збереження навченої моделі для подальшого використання  
**Навіщо:** Можна використати модель пізніше без повторного навчання


In [ ]:
# Збереження моделі
model.save('cifar10_classification_model.h5')
print("Модель збережено у файл: cifar10_classification_model.h5")
print(f"Фінальна точність моделі: {test_accuracy*100:.2f}%")

# Підсумок проекту
print("\n" + "="*50)
print("ПІДСУМОК ПРОЕКТУ")
print("="*50)
print(f"✓ Датасет: CIFAR-10 ({x_train.shape[0]} навчальних зображень)")
print(f"✓ Точність класифікації: {test_accuracy*100:.2f}%")
print(f"✓ Час обробки одного зображення: {time_per_image:.4f} сек")
print(f"✓ Модель збережена для подальшого використання")
print("="*50)
